In [1]:
#Connect to wandb
import wandb
wandb.login()
wandb.init(project="xgb", 
           entity="benchmark-nlp",
           name='sarcasm datasets') #CHANGE

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jtonglet (benchmark-nlp). Use `wandb login --relogin` to force relogin


In [2]:
import os 
os.chdir('../../..')

In [3]:
#Load packages
import warnings
import io
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker
import yaml
from util.dataloader import DataLoader
from preprocessing.preprocessor import Preprocessor
from util.datasplitter import data_splitter
from preprocessing.fasttext_embeddings import FastTextEmbeddings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score
warnings.filterwarnings("ignore")

In [4]:
#Set constants
SEED=42
OPT_ITER=20

## Load data

In [8]:
dl = DataLoader(['sarcasm'])
data = dl.load()


tweet_preprocessor = Preprocessor(is_tweet=True)
preprocessor = Preprocessor()

#We are not interested in the test sets for hyperparameter optimization
train_semeval_A, val_semeval_A, _ = data_splitter(data['SemEval_A'],
                                 tweet_preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)

train_isarcasm, val_isarcasm, _ = data_splitter(data['iSarcasm'],
                                 tweet_preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                 seed=SEED)

3817 rows preprocessed in 2.9911282062530518 seconds
784 rows preprocessed in 0.16309666633605957 seconds
3468 rows preprocessed in 0.7899963855743408 seconds
1400 rows preprocessed in 0.283416748046875 seconds


In [5]:
train_sarc, val_sarc, _ = data_splitter(data['sarc'],
                                 preprocessor, 
                                 create_val_set=True,   #No validation set is provided
                                seed=SEED)

KeyboardInterrupt: 

In [9]:
%%time
#fasttext 
fasttext = FastTextEmbeddings()
fasttext.load_model('fasttext/cc.en.300.bin')

CPU times: total: 17.8 s
Wall time: 36.2 s


In [10]:
embedded_train_semeval_A = fasttext.generate_sentence_embeddings(train_semeval_A['text'])
embedded_val_semeval_A = fasttext.generate_sentence_embeddings(val_semeval_A['text'])
embedded_train_semeval_A['label'] = train_semeval_A['label'].to_list()
embedded_val_semeval_A['label'] = val_semeval_A['label'].to_list()

Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 3053/3053 [00:04<00:00, 700.10it/s]


Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████████| 764/764 [00:01<00:00, 697.92it/s]


In [11]:
embedded_train_isarcasm = fasttext.generate_sentence_embeddings(train_isarcasm['text'])
embedded_val_isarcasm = fasttext.generate_sentence_embeddings(val_isarcasm['text'])
embedded_train_isarcasm['label'] = train_isarcasm['label'].to_list()
embedded_val_isarcasm['label'] = val_isarcasm['label'].to_list()

Starting to generate sentence embeddings


100%|█████████████████████████████████████████████████████████████████████████████| 2774/2774 [00:04<00:00, 668.45it/s]


Starting to generate sentence embeddings


100%|███████████████████████████████████████████████████████████████████████████████| 694/694 [00:01<00:00, 618.07it/s]


In [ ]:
embedded_train_sarc = fasttext.generate_sentence_embeddings(train_sarc['text'])
embedded_val_sarc = fasttext.generate_sentence_embeddings(val_sarc['text'])
embedded_train_sarc['label'] = train_sarc['label'].to_list()
embedded_val_sarc['label'] = val_sarc['label'].to_list()

## Hyperopt

In [12]:
#Load the template yaml sweep config file for logistic regression
#If the value range for an hyperparameter needs to be changed, better to do it in the .yaml file than in a notebook
with open("config/xgb_sweep.yaml", 'r') as stream:
    sweep_config = yaml.safe_load(stream)

In [13]:
#The config is displayed as a nested dictionary
sweep_config

{'method': 'random',
 'entity': 'benchmark-nlp',
 'project': 'hyperopt',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'gamma': {'min': 0, 'max': 1, 'distribution': 'uniform'},
  'n_estimators': {'min': 10, 'max': 150, 'distribution': 'int_uniform'},
  'learning_rate': {'min': 0.001, 'max': 0.1, 'distribution': 'uniform'},
  'max_depth': {'min': 0, 'max': 10, 'distribution': 'int_uniform'},
  'random_state': {'value': 42}}}

####  sem_eval_A   Tf-Idf

In [ ]:
#Don't forget to name the sweep instance  
name = 'xgb_tfidf_sem_eval_A' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_tfidf(config = None,
          train=train_semeval_A, #Change here
          val=val_semeval_A): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER) #Count : number of iterations
tracker.stop()

####  Semeval A fasttext

In [ ]:
#Don't forget to name the sweep instance   
name = 'xgb_fasttext_sem_eval_A' #change here
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_fasttext(config = None,
          train=embedded_train_semeval_A, #Change here
          val=embedded_val_semeval_A): #change here
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

####  i Sarcasm


In [16]:
#Don't forget to name the sweep instance  
name = 'xgb_tfidf_isarcasm' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_tfidf(config = None,
          train=train_isarcasm, #Change here
          val=val_isarcasm): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        vec = TfidfVectorizer()
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

Create sweep with ID: yjjhh0kd
Sweep URL: https://wandb.ai/benchmark-nlp/xgb/sweeps/yjjhh0kd


[codecarbon WARNING @ 13:46:33] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 13:46:35] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: 8mptc1md with config:
wandb: 	gamma: 0.4808179108082832
wandb: 	learning_rate: 0.09486955485991944
wandb: 	max_depth: 2
wandb: 	n_estimators: 96
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.56627
AUC-PC,0.34224
accuracy,0.71326
f1 macro,0.43053


wandb: Agent Starting Run: xxcvg5t5 with config:
wandb: 	gamma: 0.9495070256331224
wandb: 	learning_rate: 0.03283234873044642
wandb: 	max_depth: 0
wandb: 	n_estimators: 59
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.5
AUC-PC,0.28674
accuracy,0.71326
f1 macro,0.41632


...

wandb: Agent Starting Run: ibhiquyb with config:
wandb: 	gamma: 0.9334088158667438
wandb: 	learning_rate: 0.06611383767949078
wandb: 	max_depth: 8
wandb: 	n_estimators: 43
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.57186
AUC-PC,0.3335
accuracy,0.71037
f1 macro,0.44688


wandb: Agent Starting Run: 1631y1vl with config:
wandb: 	gamma: 0.2102759142809254
wandb: 	learning_rate: 0.049800778527698514
wandb: 	max_depth: 6
wandb: 	n_estimators: 35
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.55937
AUC-PC,0.32872
accuracy,0.70749
f1 macro,0.43709


0.0020891967381379477

In [17]:
#Don't forget to name the sweep instance   
name = 'xgb_ft_isarcasm' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_fasttext(config = None,
          train=embedded_train_isarcasm, #CHANGE HERE
          val=embedded_val_isarcasm): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()

[codecarbon WARNING @ 14:00:34] No CPU tracking mode found. Falling back on CPU constant mode.


Create sweep with ID: pznefzco
Sweep URL: https://wandb.ai/benchmark-nlp/xgb/sweeps/pznefzco


[codecarbon WARNING @ 14:00:37] We saw that you have a Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz but we don't know it. Please contact us.
wandb: Agent Starting Run: n6jl8eg1 with config:
wandb: 	gamma: 0.3573933324788726
wandb: 	learning_rate: 0.07136936766899564
wandb: 	max_depth: 10
wandb: 	n_estimators: 92
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.57797
AUC-PC,0.37145
accuracy,0.71758
f1 macro,0.45014


wandb: Agent Starting Run: zzp7tw68 with config:
wandb: 	gamma: 0.6329361642699473
wandb: 	learning_rate: 0.030453974056918538
wandb: 	max_depth: 2
wandb: 	n_estimators: 43
wandb: 	random_state: 42


...

wandb: Agent Starting Run: dbpjr2cs with config:
wandb: 	gamma: 0.8764105497183883
wandb: 	learning_rate: 0.0810929316558163
wandb: 	max_depth: 5
wandb: 	n_estimators: 49
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.58933
AUC-PC,0.37732
accuracy,0.71182
f1 macro,0.43449


wandb: Agent Starting Run: l5ei73re with config:
wandb: 	gamma: 0.5615850141182016
wandb: 	learning_rate: 0.09383609798968869
wandb: 	max_depth: 5
wandb: 	n_estimators: 117
wandb: 	random_state: 42


AUC,▁
AUC-PC,▁
accuracy,▁
f1 macro,▁
AUC,0.59088
AUC-PC,0.40401
accuracy,0.7219
f1 macro,0.47289


0.002386086947879153

#### sarc

In [ ]:
#Don't forget to name the sweep instance  
name = 'xgb_tfidf_sarc' #change here
sweep_config['name'] =  name

#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_tfidf(config = None,
          train=train_sarc, #Change here
          val=val_sarc): #change here
    '''
    Generic WandB function to conduct hyperparameter optimization with tf-idf vectorizer
    '''
    # Initialize a new wandb run
    with wandb.init(config=config,group=name):
        config = wandb.config
        vec = TfidfVectorizer()
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        
        #Create the pipeline
        pipe = Pipeline([('vectorizer',vec),('clf',clf)])
        #Fit the pipeline
        pipe.fit(train['text'],train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val['text'])
        pred_prob_val = pipe.predict_proba(val['text'])[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
        else:
            aucpc = '-'
            auc = '-'
        #Log metrics on WandB
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })

#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_tfidf,count=OPT_ITER)
tracker.stop()

In [ ]:
#Don't forget to name the sweep instance   
name = 'xgb_ft_sarc' #CHANGE HERE
sweep_config['name'] = name
#Generate a sweep_id
sweep_id = wandb.sweep(sweep_config, project="xgb")

def train_fasttext(config = None,
          train=embedded_train_sarc, #CHANGE HERE
          val=embedded_val_sarc): #CHANGE HERE
    # Initialize a new wandb run
    with wandb.init(config=config, group=name):
        config = wandb.config
        clf = XGBClassifier(gamma=config.gamma,
                                 n_estimators=config.n_estimators,
                                 learning_rate=config.learning_rate,
                                 max_depth=config.max_depth,
                                 random_state=config.random_state) #set the hyperparams here
        pipe = Pipeline([('clf',clf)])
        pipe.fit(train.fillna(0).drop(['label'],axis=1),train['label'])
        
        #Make predictions
        pred_val = pipe.predict(val.fillna(0).drop(['label'],axis=1))
        pred_prob_val = pipe.predict_proba(val.fillna(0).drop(['label'],axis=1))[:,1]
        accuracy = accuracy_score(val['label'],pred_val)
        f1_macro = f1_score(val['label'],pred_val,average='macro')
        if train['label'].nunique() <=2:
            aucpc =  average_precision_score(val['label'],pred_prob_val)
            auc = roc_auc_score(val['label'],pred_prob_val)
            #Log predictions on WandB
        else:
            aucpc = '-'
            auc = '-'
        wandb.log({"accuracy": accuracy, "f1 macro":f1_macro, "AUC-PC":aucpc, 'AUC':auc })


#Track emissions
tracker = EmissionsTracker(project_name=name,log_level='warning', measure_power_secs=300,
                           output_file='output/emissions_hyperopt.csv')
#Launch the agent
tracker.start()
wandb.agent(sweep_id, train_fasttext,count=OPT_ITER)
tracker.stop()